In [8]:
import quandl

In [11]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split

In [80]:
quandl.ApiConfig.api_key = 'WqusvbgLsNNWBgzbro5N'
df = quandl.get("WIKI/FB")
df.dtypes

Open           float64
High           float64
Low            float64
Close          float64
Volume         float64
Ex-Dividend    float64
Split Ratio    float64
Adj. Open      float64
Adj. High      float64
Adj. Low       float64
Adj. Close     float64
Adj. Volume    float64
dtype: object

In [17]:
import pandas

In [131]:
df = pandas.read_csv('~/AAPL.csv')
df.tail(10)

,Date,Open,High,Low,Close,Adj Close,Volume
240,2019-10-14,234.899994,238.130005,234.669998,235.869995,235.869995,24106900
241,2019-10-15,236.389999,237.649994,234.880005,235.320007,235.320007,21840000
242,2019-10-16,233.369995,235.240005,233.199997,234.369995,234.369995,18475800
243,2019-10-17,235.089996,236.149994,233.520004,235.279999,235.279999,16896300
244,2019-10-18,234.589996,237.580002,234.289993,236.410004,236.410004,24358400
245,2019-10-21,237.520004,240.990005,237.320007,240.509995,240.509995,21811800
246,2019-10-22,241.160004,242.199997,239.619995,239.960007,239.960007,20573400
247,2019-10-23,242.100006,243.240005,241.220001,243.179993,243.179993,18957200
248,2019-10-24,244.509995,244.800003,241.809998,243.580002,243.580002,17318800
249,2019-10-25,243.160004,246.729996,242.880005,246.580002,246.580002,18330500


In [117]:
df.dtypes
df = df.drop(columns='Date', axis=1)
df

,Open,High,Low,Close,Adj Close,Volume
0,219.190002,219.690002,206.089996,212.240005,208.996857,45935500
1,211.149994,215.179993,209.270004,213.300003,210.040649,36660000
2,216.880005,220.449997,216.619995,218.860001,215.515686,38358900
3,219.050003,222.360001,216.809998,222.220001,218.824341,58323200
4,209.550003,213.649994,205.429993,207.479996,204.309570,91328700
...,...,...,...,...,...,...
245,237.520004,240.990005,237.320007,240.509995,240.509995,21811800
246,241.160004,242.199997,239.619995,239.960007,239.960007,20573400
247,242.100006,243.240005,241.220001,243.179993,243.179993,18957200
248,244.509995,244.800003,241.809998,243.580002,243.580002,17318800


In [118]:
forecast_out = 3
df['Pred'] = df[['High']].shift(-forecast_out)

In [119]:
df.tail(8)

,Open,High,Low,Close,Adj Close,Volume,Pred
242,233.369995,235.240005,233.199997,234.369995,234.369995,18475800,240.990005
243,235.089996,236.149994,233.520004,235.279999,235.279999,16896300,242.199997
244,234.589996,237.580002,234.289993,236.410004,236.410004,24358400,243.240005
245,237.520004,240.990005,237.320007,240.509995,240.509995,21811800,244.800003
246,241.160004,242.199997,239.619995,239.960007,239.960007,20573400,246.729996
247,242.100006,243.240005,241.220001,243.179993,243.179993,18957200,NaN
248,244.509995,244.800003,241.809998,243.580002,243.580002,17318800,NaN
249,243.160004,246.729996,242.880005,246.580002,246.580002,18330500,NaN


In [120]:
x = np.array(df.drop(['Pred'], axis=1))

In [121]:
x = x[:-forecast_out]
# x = np.delete(x, 0, axis=1)

In [122]:
y = np.array(df['Pred']) # dependent set Y
y = y[:-forecast_out]

In [123]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [124]:
svr_conf = SVR(kernel='rbf', C=1e3, gamma=0.1)
svr_conf.fit(x_train, y_train)

SVR(C=1000.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.1,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [125]:
svm_conf = svr_conf.score(x_test, y_test)
svm_conf

-0.08465868756887884

In [126]:
# linear regression model
lr = LinearRegression()
lr.fit(x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [127]:
lr_conf = lr.score(x_test, y_test)
lr_conf

0.9230367475766182

In [128]:
x_forecast = np.array(df.drop(['Pred'], 1))[-forecast_out:]
lr_pred = lr.predict(x_forecast)
lr_pred

array([245.35154428, 245.32462206, 246.95287873])

In [129]:
svr_pred = svr_conf.predict(x_forecast)
svr_pred

array([193.19771599, 193.19771599, 193.19771599])

In [132]:
lr_pred - x_forecast

ValueError: operands could not be broadcast together with shapes (3,) (3,6) 